In [1]:
'''This notebook heavily reuses code from https://patrickwasp.com/create-your-own-coco-style-dataset/'''
'''Thank you to @waspinator for making this wonderful tool.'''


import datetime
import json
import os
import re
import fnmatch
from PIL import Image
import numpy as np
from pycococreatortools import pycococreatortools

In [31]:
DATASET_NAME = 'brcas_test'
ROOT_DIR = '/home/put_data/moth/data/COCOdataset_format/BRCAS/'
IMAGE_DIR = os.path.join(ROOT_DIR, DATASET_NAME,DATASET_NAME)
ANNOTATION_DIR = os.path.join(ROOT_DIR, DATASET_NAME,"mask")

INFO = {
    "description": "Moth Dataset_%s" %(DATASET_NAME),
    "url": "none",
    "version": "0.1.0",
    "year": 2018,
    "contributor": "KatieHYT",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "yeah",
        "url": "yeah"
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'moth',
        'supercategory': 'Moth',
    },
  
]

In [32]:
def filter_for_jpeg(root, files):
    file_types = ['*.jpeg', '*.jpg']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    
    return files

def filter_for_annotations(root, files, image_filename):
    file_types = ['*.png']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    basename_no_extension = os.path.splitext(os.path.basename(image_filename))[0]
    file_name_prefix = basename_no_extension + '.*'
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    files = [f for f in files if re.match(file_name_prefix, os.path.splitext(os.path.basename(f))[0])]

    return files

In [33]:
coco_output = {
    "info": INFO,
    "licenses": LICENSES,
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}

image_id = 1
segmentation_id = 1

# filter for jpeg images
for root, _, files in os.walk(IMAGE_DIR):
    image_files = filter_for_jpeg(root, files)

    # go through each image
    for image_filename in image_files:
        image = Image.open(image_filename)
        image_info = pycococreatortools.create_image_info(
            image_id, os.path.basename(image_filename), image.size)
        coco_output["images"].append(image_info)

        # filter for associated png annotations
        for root, _, files in os.walk(ANNOTATION_DIR):
            annotation_files = filter_for_annotations(root, files, image_filename)

            # go through each associated annotation
            for annotation_filename in annotation_files:

                print(annotation_filename)
                class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]

                category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
                binary_mask = np.asarray(Image.open(annotation_filename)
                    .convert('1')).astype(np.uint8)

                annotation_info = pycococreatortools.create_annotation_info(
                    segmentation_id, image_id, category_info, binary_mask,
                    image.size, tolerance=2)

                if annotation_info is not None:
                    coco_output["annotations"].append(annotation_info)

                segmentation_id = segmentation_id + 1

        image_id = image_id + 1
SAVE_DIR = os.path.join(ROOT_DIR,'annotations')
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
save_path = os.path.join(SAVE_DIR, 'instances_%s.json' % DATASET_NAME)

with open(save_path, 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

# will print out processed mask name


/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Lymantria_umbrifera_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Pseudabraxas_taiwana_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Himeropteryx_yui_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Calliteara_arizana_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Peridea_moorei_ochreipennis_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Pentateucha_inouei_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Calliteara_grotei_horishanella_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hyalobole_changae_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Thyas_juno_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Xestia_enyachangae_1_moth_0.png
/home/p

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Adrapsa_rivulata_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Dindica_purpurata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Simplicia_bimarginata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Psilalcis_albibasis_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Abaciscus_costimacula_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Lithophane_venusta_yazakii_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Abraxas_wilemani_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Lymantria_monacha_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Eustroma_changi_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Chandata_aglaja_1_moth_0.png
/home/put_data/mo

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ginshachia_elongata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Pseudopanolis_lala_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Orthosia_perfusca_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Theretra_alecto_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Orthosia_reticulata_fuscovestita_3_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Myrioblephara_cilicornaria_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Abraxas_antinebulosa_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Craniophora_harmandi_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Clostera_anachoreta_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Adrapsa_ablualis_1_moth_0.pn

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Simplicia_simplicissima_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Orthosia_lushana_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Eutrogia_morosa_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Sinarella_cristulalis_3_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Rikiosatoa_transversa_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Macroglossum_heliophila_heliophila_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hemithea_pallidimunda_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hypersypnoides_punctosa_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Asota_egens_indica_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Arichanna_pryeraria_4_

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Menophra_taiwana_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Agrotis_segetum_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Trachea_auriplena_auriplena_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hypopyra_vespertilio_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Magusa_interrupta_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Xestia_flavilinea_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Anorthoa_changi_6_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Psidopala_shirakii_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Mosopia_lushanensis_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Arichanna_maculosa_1_moth_0.png
/home/put_data/mot

...

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Sinarella_yuanyuan_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ourapteryx_changi_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ourapteryx_flavovirens_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Mesastrape_fulguraria_fulguraria_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Euplexidia_pallidivirens_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ecliptopera_fervidaria_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Loxaspilates_biformata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Idia_albomaculata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ramobia_anmashana_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Mesophalera_stigmatoide

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Oraesia_excavata_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Odontopera_insulata_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Pangrapta_trilineata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Simplicia_cornicalis_3_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Lysimelia_amis_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Sineugraphe_elkalandozta_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Diphtherocome_pulchra_3_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ourapteryx_flavovirens_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Mythimna_bistrigata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Antivaleria_viridentata_1_moth_0.png
/h

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Miltochrista_ziczac_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hypena_cyanea_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Thyas_coronata_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Conservula_indica_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Epilobophora_nishizawai_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Dysstroma_calamistrata_scalata_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Neodrymonia_marginalis_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Rusicada_privata_4_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Nyctemera_formosana_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Aplochlora_costipicta_1_moth_0.png
/home/

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Pseudosomera_noctuiformis_yunwu_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hadennia_hisbonalis_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Rhodostrophia_bisinuata_wilemani_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Simplicia_subrevicosta_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Rhynchaglaea_perscitula_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Hadennia_macrobliqua_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Ramobia_anmashana_3_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Albocosta_triangularis_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Abraxas_antinebulosa_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Inurois_

/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Miltochrista_takamukui_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Cidariplura_bilineata_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Microcalicha_melanosticta_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Nyctycia_strigidisca_nigridorsi_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Aedia_lichenea_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Tarsolepis_japonica_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Xylena_plumbeopaca_2_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Macrothyatira_arizana_arizana_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Alcis_rubicunda_1_moth_0.png
/home/put_data/moth/data/COCOdataset_format/BRCAS/brcas_test/mask/Pida_decolorata_macu

In [ ]:
# # don;t know why cannot directly save in annotations_dir
# ! mv ../moth/random_spread/instances_random_spread.json ../moth/random_spread/annotations/